In [1]:
!pip3 install torch==1.8.1+cpu torchvision==0.9.1+cpu torchaudio===0.8.1 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |████████████████████████████████| 169.1 MB 59 kB/s 
     |████████████████████████████████| 13.3 MB 117 kB/s 
     |████████████████████████████████| 1.9 MB 4.1 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.1+cpu which is incompatible.


In [2]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
!pip install torch-geometric
!pip install --upgrade git+https://github.com/rusty1s/pytorch_geometric.git

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
     |████████████████████████████████| 3.3 MB 4.0 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
     |████████████████████████████████| 1.9 MB 312 kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
     |████████████████████████████████| 1.1 MB 1.4 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
     |████████████████████████████████| 422 kB 4.2 MB/s 
     |████████████████████████████████| 222 kB 4.3 MB/s 
     |████████████████████████████████| 376 kB 44.5 MB/s 
     |████████████████████████████████| 45 kB 2.8 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.7.2-py3-none-any.whl size=388142 sha256=5b7b764d01a65dd7e1db9f30ba3e5df77f367874e0db50ecf58c50580e668820
  Stored in directory: /root/.cache/pip/wheels/55/93/b6/2eeb0465afe89aee74d7a07a606e9770466d7565abd45a99d5
Successfully built torch-

In [3]:
import argparse
import os.path as osp

import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.datasets import UPFD
from torch_geometric.data import DataLoader
from torch_geometric.transforms import ToUndirected
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, global_max_pool, global_mean_pool
from torch_geometric.nn import GINConv, global_add_pool,GINEConv
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU

In [18]:
path = osp.join(osp.dirname(osp.realpath("/content/tutorial1")), '..', 'data', 'UPFD')
train_dataset = UPFD(path, 'politifact', 'bert', 'train', ToUndirected())
val_dataset = UPFD(path, 'politifact', 'bert', 'val', ToUndirected())
test_dataset = UPFD(path, 'politifact', 'bert', 'test', ToUndirected())

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

Processing...
Done!


feature: profile --------ACC=77

feature: spacy----------ACC=77

feature: bert----------ACC=80

In [19]:
print(val_dataset)

UPFD(31, name=politifact, feature=bert)


In [24]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, dim, out_channels):
        super(Net, self).__init__()

        self.conv1 = GINConv(
            Sequential(Linear(in_channels, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv2 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv3 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv4 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv5 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.lin1 = Linear(dim, dim)
        self.lin2 = Linear(dim, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = self.conv4(x, edge_index)
        x = self.conv5(x, edge_index)
        x = global_add_pool(x, batch)
        x = self.lin1(x).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(train_dataset.num_features, 180, train_dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    total_correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        total_correct += int((out.argmax(-1) == data.y).sum())
    return total_correct / len(loader.dataset)


for epoch in range(1, 101):
    loss = train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f} '
          f'Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 2.9429, Train Acc: 0.5806 Test Acc: 0.4887
Epoch: 002, Loss: 1.9870, Train Acc: 0.5806 Test Acc: 0.4887
Epoch: 003, Loss: 0.9049, Train Acc: 0.5806 Test Acc: 0.4480
Epoch: 004, Loss: 1.2285, Train Acc: 0.5000 Test Acc: 0.4480
Epoch: 005, Loss: 0.7926, Train Acc: 0.5484 Test Acc: 0.4389
Epoch: 006, Loss: 0.8506, Train Acc: 0.5806 Test Acc: 0.4887
Epoch: 007, Loss: 0.8337, Train Acc: 0.5806 Test Acc: 0.4887
Epoch: 008, Loss: 0.8058, Train Acc: 0.5806 Test Acc: 0.4842
Epoch: 009, Loss: 0.7747, Train Acc: 0.5968 Test Acc: 0.4661
Epoch: 010, Loss: 0.5810, Train Acc: 0.5645 Test Acc: 0.4661
Epoch: 011, Loss: 0.7050, Train Acc: 0.5806 Test Acc: 0.4751
Epoch: 012, Loss: 0.7115, Train Acc: 0.5645 Test Acc: 0.4932
Epoch: 013, Loss: 0.6626, Train Acc: 0.5806 Test Acc: 0.4977
Epoch: 014, Loss: 0.6617, Train Acc: 0.5968 Test Acc: 0.4977
Epoch: 015, Loss: 0.5780, Train Acc: 0.5968 Test Acc: 0.5023
Epoch: 016, Loss: 0.6761, Train Acc: 0.5968 Test Acc: 0.5113
Epoch: 017, Loss: 0.5997